In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [8]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [10]:
df = pd.read_csv('/content/gdrive/My Drive/borg_traces_data.csv', index_col = 0)

In [11]:
T0 = 300000000
dT = 300000000
Tn = 2678400000000
Tsplit = 1806900000000
L = 5

In [12]:
df.sort_values(['instance_index', 'start_time'])[['start_time', 'end_time', 'machine_id', 'instance_index']]


,start_time,end_time,machine_id,instance_index
256920,600000000,900000000,378190472,0
153698,1052000000,1068000000,174861657145,0
55103,1200000000,1265000000,291250669,0
143597,1500000000,1800000000,291245817,0
392408,1800000000,2100000000,175787683989,0
...,...,...,...,...
175072,195000000000,195300000000,2205773925,170927
399990,2249100000000,2249400000000,29956815209,170952
173722,195000000000,195300000000,194848122819,171319
166309,195000000000,195300000000,19965039446,171536


In [13]:
def create_resource(string):
    try:
        dict = eval(string)
    except:
        return string
    if dict['cpus'] is None:
        return dict['memory']
    if dict['memory'] is None:
        return dict['cpus']
    return dict['cpus'] + dict['memory']

In [14]:
df['resource_req'] = df['resource_request'].map(create_resource)

In [15]:
filtered_data = df.drop(['resource_request','vertical_scaling','scheduler','cycles_per_instruction','start_after_collection_ids', 'average_usage','maximum_usage','random_sample_usage','cpu_usage_distribution','tail_cpu_usage_distribution', 'memory_accesses_per_instruction','resource_request','constraint', 'time', 'instance_events_type', 'collection_id', 'scheduling_class', 'collection_type', 'priority', 'alloc_collection_id', 'collections_events_type', 'collection_name', 'collection_logical_name', 'cluster', 'event', 'end_time'],axis=1)

In [ ]:
# filtered_data = filtered_data.sort_values(['machine_id', 'instance_index', 'start_time'])

In [16]:
def normalize(df):
    result = df.copy()
    attribute = 'resource_req'
    result[attribute] = (result[attribute] - result[attribute].min())/(result[attribute].max() - result[attribute].min())
    return result

In [17]:
for i in range(L, 0, -1):
    attribute = 'prev_request' + str(i)
    filtered_data[attribute] = filtered_data.sort_values(['start_time']).groupby(['instance_index'])['resource_req'].shift(i)

In [18]:
f_data = filtered_data.drop(['user', 'assigned_memory', 'page_cache_memory', 'sample_rate'], axis = 1)
f_data = f_data.dropna()

In [19]:
f_data = normalize(f_data)

In [20]:
train_set = f_data[(f_data['start_time'] >= T0) & (f_data['start_time'] < Tsplit)].sort_values(['instance_index'])
test_set = f_data[(f_data['start_time'] >= Tsplit) & (f_data['start_time'] < Tn)].sort_values(['instance_index'])

In [21]:
f_data.sort_values(['instance_index','start_time'])

,instance_index,machine_id,start_time,failed,resource_req,prev_request5,prev_request4,prev_request3,prev_request2,prev_request1
359851,0,300753401,2100000000,0,0.008721,0.001056,0.004164,0.009296,0.007926,0.017593
64870,0,152696973693,3000000000,0,0.009033,0.004164,0.009296,0.007926,0.017593,0.007380
18084,0,10220566,4800000000,0,0.003095,0.009296,0.007926,0.017593,0.007380,0.007644
29403,0,291336903,4800000000,0,0.003041,0.007926,0.017593,0.007380,0.007644,0.002619
224589,0,10376084361,6000000000,1,0.010887,0.017593,0.007380,0.007644,0.002619,0.002573
...,...,...,...,...,...,...,...,...,...,...
214939,64572,2314865713,2496000000000,0,0.009801,0.008293,0.008293,0.008293,0.008293,0.008293
72997,64688,28780651455,2496000000000,0,0.009801,0.008293,0.008293,0.008293,0.008293,0.008293
215290,70717,190195705866,2496000000000,0,0.009801,0.008293,0.008293,0.008293,0.008293,0.008293
367110,70717,29952997358,2496000000000,0,0.009801,0.008293,0.008293,0.008293,0.008293,0.008293


In [22]:
model = Sequential()
model.add(LSTM(100, activation = 'sigmoid', input_shape = (L, 1)))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [23]:
training = train_set.drop(['instance_index', 'machine_id', 'start_time', 'failed'], axis = 1)
# testing = test_set.drop(['instance_index', 'machine_id', 'start_time', 'failed'], axis = 1)
# training = training.dropna()
# testing = testing.dropna()

In [24]:
X_train = np.asarray(training.drop(['resource_req'], axis = 1))
y_train = np.asarray(training['resource_req'])

In [ ]:
# X_test = np.asarray(testing.drop(['resource_req'], axis = 1))
# y_test = np.asarray(testing['resource_req'])

In [25]:
model.fit(X_train, y_train)

6957/6957 [==============================] - 54s 7ms/step - loss: 0.0031


In [ ]:
# y_predict = model.predict(X_test)

In [ ]:
# y_predict

In [ ]:
# testing['pred'] = y_predict

In [ ]:
# testing[['resource_req', 'pred']]

In [ ]:
T = Tsplit
while T <= Tn:
    fd_time = test_set[(test_set['start_time'] >= T) & (test_set['start_time'] < T + dT)].sort_values(['instance_index'])
    machine_list = fd_time['machine_id'].unique().tolist()
    for machine in machine_list:
      fd_t = fd_time[fd_time['machine_id'] == machine]
      curr_data = fd_t.drop(['instance_index', 'machine_id', 'start_time', 'failed'], axis = 1)
      curr_data = curr_data.dropna()
      X_test = np.asarray(curr_data.drop(['resource_req'], axis = 1))
      y_test = np.asarray(curr_data['resource_req'])
      if X_test.size != 0:
          y_pred = model.predict(X_test)
          fd_t['res_pred'] = y_pred
          def predict_fail(row):
              if(row['prev_request1'] < row['res_pred']):
                  return 1
              return 0
          fd_t['fail_prone'] = fd_t.apply(lambda row: predict_fail(row), axis=1)
          print(fd_t[['failed','fail_prone', 'res_pred', 'resource_req']])
          model.fit(X_test, y_test)
    T += dT

1/1 [==============================] - 0s 208ms/step
       failed  fail_prone  res_pred  resource_req
91198       0           1  0.036222      0.016198
1/1 [==============================] - 0s 28ms/step
        failed  fail_prone  res_pred  resource_req
387525       0           1  0.026741      0.012154


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

1/1 [==============================] - 0s 27ms/step
        failed  fail_prone  res_pred  resource_req
257326       0           1  0.011614      0.016198
1/1 [==============================] - ETA: 0s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 28ms/step
       failed  fail_prone  res_pred  resource_req
40839       0           0 -0.002507      0.016198
1/1 [==============================] - 0s 25ms/step


...

       failed  fail_prone  res_pred  resource_req
26756       0           0  0.007241      0.013966
1/1 [==============================] - 0s 10ms/step - loss: 4.5222e-05


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 31ms/step
        failed  fail_prone  res_pred  resource_req
127126       0           1  0.008186      0.013966
1/1 [==============================] - 0s 11ms/step - loss: 3.3407e-05


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 40ms/step


In [ ]:
# y_pred